In [13]:
from py2neo import Graph,Node,Relationship

douban_KG = Graph(
    "http://localhost:7474", 
    username="neo4j", 
    password="luoruipu"
)
douban_KG

<Graph database=<Database uri='http://localhost:7474' secure=False user_agent='py2neo/4.3.0 urllib3/1.24.2 Python/3.7.4-final-0 (darwin)'> name='data'>

In [3]:
import pymongo

client = pymongo.MongoClient('127.0.0.1',port = 27017)
db = client.douban
movie_collection = db.movies
person_collection = db.celebrities
movie_result = movie_collection.find({'id':'6874741'})
person_result = person_collection.find({'id':'1041014'})
for i in movie_result:
    print(i)
print()
for i in person_result:
    print(i)

{'_id': ObjectId('5ebb7f1c529e34ec924a0fec'), 'id': '6874741', 'title': '无问西东', 'director': '1313682%李芳芳', 'editor': '1313682%李芳芳', 'actors': '1336356%保罗·菲利普·克拉克/1325511%胡家华/1313886%伊娜/1337020%吴谨言/1041404%黄晓明/1371530%纪帅/1041014%章子怡/1357288%王天泽/1077991%张震/1335184%伍麟凯/1045243%王力宏/1350175%都金翰/1316875%陈楚生/1313358%铁政/1313650%祖峰/1275431%米雪/1032117%王盛德/1274722%韩童生/1335594%王鑫/1274516%郑铮/1338842%章泽天/1349244%黄梦莹/1316086%林美秀', 'type_': '剧情/爱情/战争', 'region': '中国大陆', 'languages': '汉语普通话', 'release_date': '2018-01-12(中国大陆)', 'mins': '138分钟', 'alias': 'Forever Young', 'rating': '7.6', 'douban_votes': '592018', 'storyline': '如果提前了解了你所要面对的人生，你是否还会有勇气前来？吴岭澜、沈光耀、王敏佳、陈鹏、张果果，几个年轻人满怀诸多渴望，在四个非同凡响的时空中一路前行。吴岭澜（陈楚生饰），出发时意气风发，却很快在途中迷失了方向。沈光耀（王力宏饰），自愿参与了最残酷的战争，他一直在努力去做那些令他害怕，但重要的事。王敏佳（章子怡饰）最初的错误，只是为了虚荣撒了一个小谎；最初的烦恼，只是在两个优秀的男人中选择一个。但命运，却把她拖入被众人唾骂的深渊。陈鹏（黄晓明饰）把爱情摆在了理想前面，但爱情却没有把他摆在前面。他说，“我有人要照顾”，纵然这意味着与所有人作对，意味着要和她一起被放逐千里。张果果（张震饰），身处尔虞我诈的职场，“赢”是他的习惯。为了赢，他总是见招拆招，先发制人。而有一天，他却面临了一个比“赢”更重要的选择。这几个年轻人，在最好的年纪迎来了最残酷的...'}



In [15]:
movie_result = movie_collection.find({'id':'27024903'})
for i in movie_result:
    createMovieNode(douban_KG,i)

In [46]:
douban_KG.delete_all()

In [51]:
createMovieNode(douban_KG,dic)

爱情
战争


In [5]:
def createMovieNode(graph,movie_dictory):
    movie_node = Node("电影",\
        name = movie_dictory['title']
        )
    movie_node['地区'] = movie_dictory['region']
    movie_node['语言'] = movie_dictory['languages']
    movie_node['发行日期'] = movie_dictory['release_date']
    movie_node['片长'] = movie_dictory['mins']
    movie_node['别名'] = movie_dictory['alias']
    movie_node['评分'] = movie_dictory['rating']
    movie_node['打分人数'] = movie_dictory['douban_votes']
    movie_node['故事简介'] = movie_dictory['storyline']
    graph.create(movie_node)

    type_list = movie_dictory['type_'].split('/')
    for typ in type_list:
        type_node = douban_KG.nodes.match('类型', name=typ).first()
        
        if not type_node:
            type_node = Node("类型",name = typ)
            graph.create(type_node)
        graph.create(Relationship(movie_node,'属于',type_node))

    director_list = movie_dictory['director'].split('/')
    for x  in director_list:
        xid = x.split('%')[0]
        xname = x.split('%')[1]
        director_node = douban_KG.nodes.match('影人', name=xname).first()
        if not director_node:
            director_node = Node('影人',name = xname) 
            graph.create(director_node)
        graph.create(Relationship(director_node,'导演',movie_node))

    actor_list = movie_dictory['actors'].split('/')
    for x  in actor_list:
        xid = x.split('%')[0]
        xname = x.split('%')[1]
        actor_node = douban_KG.nodes.match('影人', name=xname).first()
        if not actor_node:
            actor_node = Node('影人',name = xname) 
            graph.create(actor_node)
        graph.create(Relationship(actor_node,'参演',movie_node))
    
    editor_list = movie_dictory['editor'].split('/')
    for x  in editor_list:
        xid = x.split('%')[0]
        xname = x.split('%')[1]
        editor_node = douban_KG.nodes.match('影人', name=xname).first()
        if not editor_node:
            editor_node = Node('影人',name = xname) 
            graph.create(editor_node)
        graph.create(Relationship(editor_node,'编剧',movie_node))

    

In [14]:
type(douban_KG.nodes.match('类型', name='喜剧').first())


NoneType

[5, 15, 22, 30]